In [ ]:
!nvidia-smi

Wed Oct 25 03:55:58 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-a4xm_4ec
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-a4xm_4ec
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=41c85afe4f88d86955e9b979f7c7b107d46a1036118947108cf47b33b4fc1501
  Stored in directory: /tmp/pip-ephem-wheel-cache-h5i9suyt/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
//Program 1 - Matrix Addition, Not sure if there for CIA-1 LAB or not

#include<stdio.h>
#include<cuda.h>
// Define the no of rows as nx, columns as ny below....
#define nx 4
#define ny 10

__global__ void madd(int *a, int *b, int *c)
{
    int ix=threadIdx.x + blockIdx.x*blockDim.x;
    int iy=threadIdx.y + blockIdx.y*blockDim.y;
    int linear_address = iy*nx + ix;
    c[linear_address]=a[linear_address]+b[linear_address];
}

int main()
{
    int *a, *b, *c;
    int i;
    int ix, iy;
    int size = nx * ny * sizeof(int);
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    int *d_a, *d_b, *d_c;
    cudaMalloc((void**)&d_a, size);
    cudaMalloc((void**)&d_b, size);
    cudaMalloc((void**)&d_c, size);


    for(i=0; i<nx*ny; i++)
    {
        a[i] = i;
        b[i] = i+1;
    }
    printf("Matrix A \n");

    for(ix=0; ix<nx; ix++)
    {
        for(iy=0; iy<ny; iy++)
        {
            printf("\t %d", a[iy+ix*ny]);
        }
        printf("\n");
    }

    printf("Matrix B \n");

    for(ix=0; ix<nx; ix++)
    {
        for(iy=0; iy<ny; iy++)
        {
            printf("\t %d", b[iy+ix*ny]);
        }
        printf("\n");
    }


    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    dim3 block(2,2);
    dim3 grid((nx+block.x-1)/block.x,(ny+block.y-1)/block.y);

    madd<<<grid, block>>>(d_a, d_b, d_c);

    cudaDeviceSynchronize();
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    printf("\n\n Destination Matrix C \n\n");

    for(ix=0; ix<nx; ix++)
    {
        for(iy=0; iy<ny; iy++)
        {
            printf("\t %d", c[iy+ix*ny]);
        }
        printf("\n");
    }




    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    free(a);
    free(b);
    free(c);
    cudaDeviceReset();
    return 0;
}

Matrix A 
	 0	 1	 2	 3	 4	 5	 6	 7	 8	 9
	 10	 11	 12	 13	 14	 15	 16	 17	 18	 19
	 20	 21	 22	 23	 24	 25	 26	 27	 28	 29
	 30	 31	 32	 33	 34	 35	 36	 37	 38	 39
Matrix B 
	 1	 2	 3	 4	 5	 6	 7	 8	 9	 10
	 11	 12	 13	 14	 15	 16	 17	 18	 19	 20
	 21	 22	 23	 24	 25	 26	 27	 28	 29	 30
	 31	 32	 33	 34	 35	 36	 37	 38	 39	 40


 Destination Matrix C 

	 1	 3	 5	 7	 9	 11	 13	 15	 17	 19
	 21	 23	 25	 27	 29	 31	 33	 35	 37	 39
	 41	 43	 45	 47	 49	 51	 53	 55	 57	 59
	 61	 63	 65	 67	 69	 71	 73	 75	 77	 79




 Array A 
0 1 2 3 4 5 6 7 8 9 0 1 
2 3 
0 1 
2 3 
1 5 
2 8 
10 4 
18 4 


 Destination Array A 

26 1 8 3 4 5 6 7 8 9 


In [ ]:
%%cu
#include<stdio.h>
#include<cuda.h>
#include<math.h>
#define N 8
// GPT CIA-1 Parallel Reduction Sum of Array
__global__ void parallel(int* a)
{
  int tid=threadIdx.x;
  for(int j=0;j<=log2f(N)-1;j++)
  {
    float z=powf(2,j);
    int x=tid%(int)z;
    int y=powf(2,j);

    if( x==0 && ((2*tid+y) < N) )
    {
      a[2*tid] = a[2*tid] + a[2*tid + y];
    }
  }
}

int main()
{
  int *a;
  int size=N*sizeof(int);
  a=(int *)malloc(size);
  int *d_a;
  cudaMalloc((void**)&d_a,size);
  for(int i=0;i<N;i++)
  {
    a[i]=i+1;
  }
  printf("Elements of A \n");
  for(int i=0;i<N;i++)
  {
    printf(" %d ",a[i]);
  }
  printf("\n");
  cudaMemcpy(d_a,a,size,cudaMemcpyHostToDevice);

  parallel<<<(N/2),(N/2)>>>(d_a);

  cudaMemcpy(a,d_a,size,cudaMemcpyDeviceToHost);

  for(int i=0;i<N;i++)
  {
    printf("%d ",a[i]);
  }

  cudaFree(d_a);
  return 0;
}


Enter elements of A: Elements of A 
 1  2  3  4  5  6  7  8 
36 2 7 4 26 6 15 8 



 Enter N value:Give  Input Nos. : 


 Sorted array:
Enter k for finding the kth largest no. :

The kth largest no. is : 0


1 2 3 4 5 6 7 8 9 10 Thread Indices 6 9 
Thread Indices 7 9 
Thread Indices 8 9 
Thread Indices 9 9 
Thread Indices 2 3 
Thread Indices 3 3 
Thread Indices 4 3 
Thread Indices 5 3 
Thread Indices 6 3 
Thread Indices 7 3 
Thread Indices 8 3 
Thread Indices 9 3 
Thread Indices 0 4 
Thread Indices 1 4 
Thread Indices 2 4 
Thread Indices 3 4 
Thread Indices 4 4 
Thread Indices 5 4 
Thread Indices 6 4 
Thread Indices 7 4 
Thread Indices 8 4 
Thread Indices 9 4 
Thread Indices 0 5 
Thread Indices 1 5 
Thread Indices 2 5 
Thread Indices 3 5 
Thread Indices 4 5 
Thread Indices 5 5 
Thread Indices 6 5 
Thread Indices 7 5 
Thread Indices 8 5 
Thread Indices 9 5 
Thread Indices 0 6 
Thread Indices 1 6 
Thread Indices 2 6 
Thread Indices 3 6 
Thread Indices 4 6 
Thread Indices 5 6 
Thread Indices 6 6 
Thread Indices 7 6 
Thread Indices 8 6 
Thread Indices 9 6 
Thread Indices 0 7 
Thread Indices 1 7 
Thread Indices 2 7 
Thread Indices 3 7 
Thread Indices 4 7 
Thread Indices 5 7 
Thread Indices 6 7 

In [ ]:
%%cu
#include <cuda_runtime.h>
#include <stdio.h>
#define N 10
// Enumeration Sort
__global__ void sort(int *p, int *a, int *s)
{
    int ix = threadIdx.x;
    p[ix] = N-1;

    for (int iy = 0; iy < N; iy++)
    {
        if (a[ix] < a[iy] || (a[ix] == a[iy] && ix < iy))
        {
            p[ix] -= 1;
        }
    }

    __syncthreads();

    if (ix < N)
    {
        s[p[ix]] = a[ix];
    }
}

int main()
{
    int *p, *a, *s;

    int size = N * sizeof(int);
    a = (int *)malloc(size);
    p = (int *)malloc(size);
    s = (int *)malloc(size);

    int i;

    int *d_p, *d_a, *d_s;
    cudaMalloc((void **)&d_p, size);
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_s, size);

    a[0]=100;
    a[1]=20;
    a[3]=50;
    for (i = 4; i < N; i++)
    {
        a[i] = i+1; // Reverse order for sorting
        p[i] = 0;
    }

    for (i = 0; i < N; i++)
    {
        printf("%d ", a[i]);
    }

    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_p, p, size, cudaMemcpyHostToDevice);

    sort<<<1, N>>>(d_p, d_a, d_s);

    cudaMemcpy(s, d_s, size, cudaMemcpyDeviceToHost);

    printf("\n\n Sorted array:\n");
    for (i = 0; i < N; i++)
        printf("%d ", s[i]);

    // Free allocated memory
    free(a);
    free(p);
    free(s);
    cudaFree(d_a);
    cudaFree(d_p);
    cudaFree(d_s);

    return 0;
}


100 20 0 50 5 6 7 8 9 10 

 Sorted array:
0 5 6 7 8 9 10 20 50 100 


In [ ]:
%%cu

#include<stdio.h>
#define N 8
// Prefix Sum
__global__ void prefixSum(int* a)
{
  int tid=threadIdx.x;
  for(int j=0;j<=log2f(N)-1;j++)
  {
    float z=powf(2,j);
    int x = tid-(int)z;

    if(x>=0)
    {
      a[tid] = a[tid] + a[x];
    }
  }
}

int main()
{
  int *a;
  int size=N*sizeof(int);
  a=(int *)malloc(size);
  int *d_a;
  cudaMalloc((void**)&d_a,size);
  //printf("\nEnter elements of A: ");
  for(int i=0;i<N;i++)
  {
    a[i]=i+1;
  }
  cudaMemcpy(d_a,a,size,cudaMemcpyHostToDevice);

  prefixSum<<<N,N>>>(d_a);

  cudaMemcpy(a,d_a,size,cudaMemcpyDeviceToHost);

  for(int i=0;i<N;i++)
  {
    printf("%d ",a[i]);
  }

  cudaFree(d_a);
  return 0;
}

1 3 6 10 15 21 28 36 


Result: 2 6 12 20 30 42 56 72 90 110 



In [ ]:
%%cu
#include <stdio.h>
// GCD
__device__ int gcd(int a, int b) {
    while (b != 0) {
        int temp = b;
        b = a % b;
        a = temp;
    }
    return a;
}

__global__ void gcdKernel(int* vec1, int* vec2, int* result, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < size) {
        int a = vec1[idx];
        int b = vec2[idx];
        result[idx] = gcd(a, b);
    }
}

int main() {
    int n = 10;
    int *h_vec1, *h_vec2;
    int *d_vec1, *d_vec2;
    int *h_result, *d_result;

    h_vec1 = (int*)malloc(n * sizeof(int));
    h_vec2 = (int*)malloc(n * sizeof(int));
    h_result = (int*)malloc(n * sizeof(int));

    cudaMalloc((void**)&d_vec1, n * sizeof(int));
    cudaMalloc((void**)&d_vec2, n * sizeof(int));
    cudaMalloc((void**)&d_result, n * sizeof(int));

    for (int i = 0; i < n; i++) {
        h_vec1[i] = i+4;
        h_vec2[i] = i+8;
    }

    cudaMemcpy(d_vec1, h_vec1, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_vec2, h_vec2, n * sizeof(int), cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    gcdKernel<<<blocksPerGrid, threadsPerBlock>>>(d_vec1, d_vec2, d_result, n);

    cudaMemcpy(h_result, d_result, n * sizeof(int), cudaMemcpyDeviceToHost);

    printf("GCD Result: ");
    for (int i = 0; i < n; i++) {
        printf("%d ", h_result[i]);
    }
    printf("\n");

    free(h_vec1);
    free(h_vec2);
    free(h_result);
    cudaFree(d_vec1);
    cudaFree(d_vec2);
    cudaFree(d_result);

    return 0;
}


GCD Result: 4 1 2 1 4 1 2 1 4 1 



In [ ]:
%%cu
#include <stdio.h>
// LCM
__device__ int gcd(int a, int b) {
    while (b != 0) {
        int temp = b;
        b = a % b;
        a = temp;
    }
    return a;
}

__device__ int lcm(int a, int b) {
    return (a * b) / gcd(a, b);
}

__global__ void lcmKernel(int* vec1, int* vec2, int* result, int size) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < size) {
        int a = vec1[idx];
        int b = vec2[idx];
        result[idx] = lcm(a, b);
    }
}

int main() {
    int n = 10;
    int *h_vec1, *h_vec2;
    int *d_vec1, *d_vec2;
    int *h_result, *d_result;

    h_vec1 = (int*)malloc(n * sizeof(int));
    h_vec2 = (int*)malloc(n * sizeof(int));
    h_result = (int*)malloc(n * sizeof(int));

    cudaMalloc((void**)&d_vec1, n * sizeof(int));
    cudaMalloc((void**)&d_vec2, n * sizeof(int));
    cudaMalloc((void**)&d_result, n * sizeof(int));

    for (int i = 0; i < n; i++) {
        h_vec1[i] = i + 4;
        h_vec2[i] = i + 8;
    }


    cudaMemcpy(d_vec1, h_vec1, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_vec2, h_vec2, n * sizeof(int), cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (n + threadsPerBlock - 1) / threadsPerBlock;
    lcmKernel<<<blocksPerGrid, threadsPerBlock>>>(d_vec1, d_vec2, d_result, n);

    cudaMemcpy(h_result, d_result, n * sizeof(int), cudaMemcpyDeviceToHost);

    printf("LCM Result: ");
    for (int i = 0; i < n; i++) {
        printf("%d & %d = %d \n ", h_vec1[i],h_vec2[i],h_result[i]);
    }
    printf("\n");

    free(h_vec1);
    free(h_vec2);
    free(h_result);
    cudaFree(d_vec1);
    cudaFree(d_vec2);
    cudaFree(d_result);

    return 0;
}


LCM Result: 4 & 8 = 8 
 5 & 9 = 45 
 6 & 10 = 30 
 7 & 11 = 77 
 8 & 12 = 24 
 9 & 13 = 117 
 10 & 14 = 70 
 11 & 15 = 165 
 12 & 16 = 48 
 13 & 17 = 221 
 



In [ ]:
// Suffix Sum not yet completed

SyntaxError: ignored